In [1]:
import os
import time
import hashlib
import numpy as np
import pandas as pd
from PIL import Image
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import MultiLabelBinarizer

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import torchvision.transforms as transforms
import timm

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


/student/sidd1091/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Load CSV data
df = pd.read_csv("/student/csc490_project/shared/labels.csv")

# Define the 15 classes (14 diseases plus "No Finding")
CLASSES = [
    "No Finding", "Atelectasis", "Cardiomegaly", "Effusion", "Infiltration", 
    "Mass", "Nodule", "Pneumonia", "Pneumothorax", "Consolidation", 
    "Edema", "Emphysema", "Fibrosis", "Pleural_Thickening", "Hernia"
]

# Process the "Finding Labels" column (split by '|' so that "No Finding" becomes ["No Finding"])
def process_labels(label_str):
    return label_str.split("|")

df["label_list"] = df["Finding Labels"].apply(process_labels)

# Initialize MultiLabelBinarizer with the defined CLASSES
mlb = MultiLabelBinarizer(classes=CLASSES)
labels_array = mlb.fit_transform(df["label_list"])

# Save the one-hot encoded labels into a new column
df["labels"] = list(labels_array)

# Print some samples to verify order
print("MultiLabelBinarizer class order:", mlb.classes_)
print("First few processed labels (raw):", df["label_list"].head().tolist())
print("First few processed labels (one-hot):", df["labels"].head().tolist())


MultiLabelBinarizer class order: ['No Finding' 'Atelectasis' 'Cardiomegaly' 'Effusion' 'Infiltration'
 'Mass' 'Nodule' 'Pneumonia' 'Pneumothorax' 'Consolidation' 'Edema'
 'Emphysema' 'Fibrosis' 'Pleural_Thickening' 'Hernia']
First few processed labels (raw): [['Cardiomegaly'], ['Cardiomegaly', 'Emphysema'], ['Cardiomegaly', 'Effusion'], ['No Finding'], ['Hernia']]
First few processed labels (one-hot): [array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]), array([0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1])]


In [ ]:
# Compose training transforms: apply augmentation, then convert to 3 channels and apply other augmentations.
train_transforms_list = []
train_transforms_list += [
    transforms.Grayscale(num_output_channels=3),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
]
train_transform = transforms.Compose(train_transforms_list)

# Validation transforms: deterministic preprocessing only
val_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])


In [5]:
class ChestXrayDataset(Dataset):
    def __init__(self, df, root_dir, transform=None):
        """
        Args:
            df: DataFrame with "Image Index" and "labels" columns.
            root_dir: Directory where images are stored.
            transform: Transformations to apply.
        """
        self.df = df
        self.root_dir = root_dir
        self.transform = transform
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.df.iloc[idx]["Image Index"])
        image = Image.open(img_name).convert("L")  # load in grayscale
        
        # Get the one-hot label vector
        labels = np.array(self.df.iloc[idx]["labels"], dtype=np.float32)
        labels = torch.tensor(labels, dtype=torch.float)
        
        if self.transform:
            image = self.transform(image)
        return image, labels

In [ ]:
def train_and_evaluate(model_name, train_df, val_df, epochs=7, batch_size=16):
    # Adjust epochs for specific models
    if model_name == 'convnext_large.fb_in22k':
        epochs = 4

    # Create model using timm
    model = timm.create_model(model_name, pretrained=True, num_classes=len(CLASSES))
    model = model.to(device)
    
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-4)
    
    # Create DataLoaders
    train_dataset = ChestXrayDataset(train_df, 
                                     root_dir="/student/csc490_project/shared/preprocessed_images/preprocessed_images", 
                                     transform=train_transform)
    val_dataset = ChestXrayDataset(val_df, 
                                   root_dir="/student/csc490_project/shared/preprocessed_images/preprocessed_images", 
                                   transform=val_transform)
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
    
    # Training loop
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * images.size(0)
        epoch_loss = running_loss / len(train_dataset)
        print(f"Model: {model_name} | Epoch {epoch+1}/{epochs} | Training Loss: {epoch_loss:.4f}")
    
    # Evaluation loop
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            preds = torch.sigmoid(outputs).cpu().numpy()
            all_preds.append(preds)
            all_labels.append(labels.cpu().numpy())
    
    all_preds = np.concatenate(all_preds, axis=0)
    all_labels = np.concatenate(all_labels, axis=0)
    
    # Compute AUROC for each class
    auroc_per_class = {}
    for i, class_name in enumerate(CLASSES):
        try:
            auroc = roc_auc_score(all_labels[:, i], all_preds[:, i])
        except ValueError:
            auroc = float('nan')
        auroc_per_class[class_name] = auroc
    overall_auroc = np.nanmean(list(auroc_per_class.values()))
    
    return model, auroc_per_class, overall_auroc

In [7]:
model_name = 'densenet121'

In [ ]:
from sklearn.model_selection import KFold
import numpy as np

# Extract unique patient IDs
unique_patients = df["Patient ID"].unique()

# Shuffle them for reproducible random folds
np.random.seed(42)
np.random.shuffle(unique_patients)

# KFold on patient IDs: 4 folds
n_splits = 4
kf = KFold(n_splits=n_splits, shuffle=False)  # We already shuffled above

# Create a list of (train_pat_idx, val_pat_idx) for each fold
folds = list(kf.split(unique_patients))

# Containers for per-fold results
fold_overall_scores = []
fold_class_scores = []
fold_models = []

for fold_idx, (train_pat_idx, val_pat_idx) in enumerate(folds):
    print(f"\n--- Fold {fold_idx+1} ---")
    
    # Identify patient IDs for training vs. validation
    train_patients = unique_patients[train_pat_idx]
    val_patients = unique_patients[val_pat_idx]
    
    # Print a few patient IDs to verify the split
    print(f"Train patient IDs (first 5): {train_patients[:5]}")
    print(f"Val patient IDs (first 5): {val_patients[:5]}")
    
    # Filter the main DataFrame by these patient IDs
    train_fold_df = df[df["Patient ID"].isin(train_patients)].reset_index(drop=True)
    val_fold_df = df[df["Patient ID"].isin(val_patients)].reset_index(drop=True)
    
    # Train and evaluate the model on this fold
    model, auroc_per_class, overall_auroc = train_and_evaluate(
        model_name, 
        train_fold_df, 
        val_fold_df, 
        epochs=7, 
        batch_size=16
    )
    
    # Save results
    fold_models.append(model)
    fold_class_scores.append(auroc_per_class)
    fold_overall_scores.append(overall_auroc)
    
    # Print fold results
    print("Per-class AUROC for this fold:", auroc_per_class)
    print("Overall AUROC for this fold:", overall_auroc)
    
    # Save the model for this fold
    fold_model_save_path = f"/student/csc490_project/shared/cv_no_overlap/{model_name}_fold{fold_idx+1}_model.pth"
    torch.save(model.state_dict(), fold_model_save_path)
    print(f"Model for Fold {fold_idx+1} saved to {fold_model_save_path}")



--- Fold 1 ---
Train indices (first 5): [0 1 2 4 5]
Val indices (first 5): [ 3 23 24 31 35]
Model: densenet121 | Epoch 1/7 | Training Loss: 0.1928
Model: densenet121 | Epoch 2/7 | Training Loss: 0.1798
Model: densenet121 | Epoch 3/7 | Training Loss: 0.1755
Model: densenet121 | Epoch 4/7 | Training Loss: 0.1721
Model: densenet121 | Epoch 5/7 | Training Loss: 0.1696
Model: densenet121 | Epoch 6/7 | Training Loss: 0.1671
Model: densenet121 | Epoch 7/7 | Training Loss: 0.1646
Per-class AUROC for this fold: {'No Finding': np.float64(0.7850596914163452), 'Atelectasis': np.float64(0.8255115199272854), 'Cardiomegaly': np.float64(0.9089117505625448), 'Effusion': np.float64(0.8842219352107729), 'Infiltration': np.float64(0.7138223566738889), 'Mass': np.float64(0.8726403247451613), 'Nodule': np.float64(0.767122030701731), 'Pneumonia': np.float64(0.7622452822068728), 'Pneumothorax': np.float64(0.8939236911411761), 'Consolidation': np.float64(0.8042551312304459), 'Edema': np.float64(0.905105064297

In [ ]:
# Compute final metrics across folds
avg_overall_auroc = np.mean(fold_overall_scores)
std_overall_auroc = np.std(fold_overall_scores)

print("\n=== Final Cross Validation Results ===")
for i, score in enumerate(fold_overall_scores):
    print(f"Fold {i+1} Overall AUROC: {score:.4f}")
print(f"Average Overall AUROC: {avg_overall_auroc:.4f}")
print(f"Standard Deviation of Overall AUROC: {std_overall_auroc:.4f}")

# Compute average per-class AUROC
avg_class_auroc = {}
for class_name in CLASSES:
    class_scores = [fold[class_name] for fold in fold_class_scores]
    avg_class_auroc[class_name] = np.nanmean(class_scores)
print("Average Per-class AUROC:", avg_class_auroc)


=== Final Cross Validation Results ===
Fold 1 Overall AUROC: 0.8371
Fold 2 Overall AUROC: 0.8373
Fold 3 Overall AUROC: 0.8377
Fold 4 Overall AUROC: 0.8380
Fold 5 Overall AUROC: 0.8408
Average Overall AUROC: 0.8382
Standard Deviation of Overall AUROC: 0.0014
Average Per-class AUROC: {'No Finding': np.float64(0.7848549510416728), 'Atelectasis': np.float64(0.8215454656265969), 'Cardiomegaly': np.float64(0.9085123253802798), 'Effusion': np.float64(0.8853314299877846), 'Infiltration': np.float64(0.7190182817430568), 'Mass': np.float64(0.8630678068345567), 'Nodule': np.float64(0.7703859677545722), 'Pneumonia': np.float64(0.7611108592699548), 'Pneumothorax': np.float64(0.8970538237922051), 'Consolidation': np.float64(0.8092851842277943), 'Edema': np.float64(0.902735975600679), 'Emphysema': np.float64(0.9219252702920757), 'Fibrosis': np.float64(0.8155751268871493), 'Pleural_Thickening': np.float64(0.8025659850242336), 'Hernia': np.float64(0.9096896575245127)}
